# WIKIPEDIA API
### Data Integration and Analysis Preparation
This Jupyter notebook is dedicated to fetching and integrating full text data from Wikipedia's API and merging it with existing datasets. Our goal is to enrich our data with additional Wikipedia-sourced information, enabling a more comprehensive analysis of artists in our dataset. This process includes data fetching, cleaning, and merging stages.

# Import Necessary Libraries and Load Data
Here, we import the Python libraries necessary for data manipulation, web requests, and visualization. We also load the initial datasets that will be used in the data merging process.

In [2]:
import requests
import json
import pandas as pd
from datetime import datetime

In [ ]:
#with open('/Users/vincenzocamerlengo/Desktop/Data Visualization/Project/Cartella Condivisa/people_names.json', 'r') as file_to_read:
    #people_names=json.load(file_to_read)

df = pd.read_csv('/Users/vincenzocamerlengo/Desktop/Data Visualization/Project/Cartella Condivisa/wordlcoud/Artists_Genres.csv')

In [ ]:
df

,Artist,Genre
0,Taylor Swift,pop music
1,XXXTentacion,punk rock
2,Alia Bhatt,Unspecified
3,Tina Turner,rock and roll
4,Keanu Reeves,hip hop music
...,...,...
991,Heather Headley,soul music
992,Crowded House,alternative rock
993,Hwasa,jazz
994,CeCe Winans,gospel music


# Fetching Data Using Wikipedia's API
In this section, we utilize Wikipedia's API to retrieve full texts relevant to the artists in our dataset. We handle web requests and parse the returned JSON data into a structured format for further processing.

In [ ]:
base_url = "https://en.wikipedia.org/w/api.php"
session = requests.Session()

headers={'User-Agent': 'Gian/1.0 (gianlorenz.marchioni@studenti.luiss.it)'}

data=[]

article_found_counter = 0

for name in df['Artist']:
       
    #print(f"Searching for: {name}")

    # Attempt to find by title
    params = {
        "action": "query",
        "format": "json",
        "titles": name,
        "prop": "extracts",
        "exlimit": 1,
        "explaintext": True,  # Get plain text instead of HTML
        "pllimit": "max"
    }
    response_full_text = session.get(url=base_url, params=params)
    #print(response_full_text.status_code)
    #print(response_full_text.text)
    pages_full_text = response_full_text.json().get("query", {}).get("pages", {})
    page_id = next(iter(pages_full_text))
    if page_id != "-1":
        article_found_counter+=1
        page = pages_full_text[page_id]
        #print(f"Found by title: {page['title']} - {page.get('fullurl', 'URL not available')}")

        page = pages_full_text[page_id]
        full_text = page.get('extract', '')

        

        # Save collected data (you might want to structure it differently based on your needs)
        data.append({
            "name": name,
            "full_text": full_text
        })
        
    else:
        print(f"No results found for {name}")

print(f"{article_found_counter} articles found" )

No results found for ZoÃ« Kravitz
No results found for BjÃ¶rk
No results found for MÃ¥neskin
No results found for Israel KamakawiwoÊ»ole
No results found for Michael BublÃ©
No results found for RenÃ© AngÃ©lil
No results found for MÃ©lanie Laurent
No results found for BjÃ¶rn Ulvaeus
No results found for GÃ©rard Depardieu
No results found for JhenÃ© Aiko
No results found for Matthias SchweighÃ¶fer
No results found for ThalÃ­a
No results found for Pedro AlmodÃ³var
No results found for Bachmanâ€“Turner Overdrive
No results found for TiÃ«sto
No results found for JosÃ© Feliciano
No results found for RaÃºl Esparza
No results found for Vicente FernÃ¡ndez
No results found for RenÃ©e Elise Goldsberry
977 articles found


In [ ]:
df1=pd.DataFrame(data)

In [ ]:
df1 = df1.rename(columns={'name': 'Artist'})

In [ ]:
df = pd.merge(df, df1, on='Artist', how='inner')

In [ ]:
#df.to_csv("/Users/vincenzocamerlengo/Desktop/Data Visualization/Project/Cartella Condivisa/wordlcoud/file_definitivo.csv")

# Data Cleaning and Standardization
Post-fetching, the data often requires cleaning which may involve removing extraneous formatting, handling missing values, and standardizing data entries for consistency across our datasets.

In [3]:
wiki = pd.read_csv("/Users/vincenzocamerlengo/Desktop/Data Visualization/Project/Cartella Condivisa/wordlcoud/file_definitivo.csv")
wiki.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
wiki[wiki.isnull().any(axis=1)]

,Artist,Genre,full_text
615,Mos Def,hip hop music,NaN
692,S Club 7,pop music,NaN


In [5]:
wiki.iloc[615, 2]

nan

In [6]:
text = ''' S Club, formerly known as S Club 7, are a British pop group formed in 1998 by Simon Fuller after he was fired as manager of the Spice Girls.[1] Original members were Tina Barrett, Paul Cattermole, Jon Lee, Bradley McIntosh, Jo O'Meara, Hannah Spearritt and Rachel Stevens. Cattermole left the group in 2002; it disbanded in 2003. After the members performed solo and in smaller groups, the group reunited temporarily for a short tour in 2015. In 2023, the group reunited and announced a tour; however, Cattermole died of heart failure and Spearritt withdrew from the group before the tour.
The band has sold more than 10 million albums worldwide.[2] They won BRIT Awards in 2000 for British breakthrough act and in 2002 for best British single. In 2001, they earned the Record of the Year award.
The band has released four albums: S Club (1999), which reached number one in the UK, 7 (2000), Sunshine (2001), and Seeing Double (2002). The band has had commercial success with songs including "Bring It All Back", "You're My Number One", "Two in a Million", "S Club Party", "Reach", "Natural", "Never Had a Dream Come True", "Don't Stop Movin'", "You", "Have You Ever", "Alive", and "Love Ain't Gonna Wait for You". In July 2023, S Club released their first new single in more than 20 years, "These Are the Days", in memory of Cattermole.1997–1998: Formation
S Club 7 were formed by Simon Fuller, after he was fired as manager of the Spice Girls in November 1997.[3] He described S Club 7 as a continuation of ideas he had had for the Spice Girls, but with a softer, more uplifting image and the same songwriting team was used for both acts.[4]
Fuller placed an advert in The Stage[5] and held auditions with more than 10,000 applicants.[6] Both Jo O'Meara and Paul Cattermole were spotted by producers from Fuller's company, 19 Management, and asked to audition.[7] Rachel Stevens was the only member who did not audition; instead, two producers approached her and asked her to record a demo tape.[7] According to journalist Steven Poole, after Fuller's disagreements with the Spice Girls, he picked "the blandest, most malleable characters ... nice kids who wouldn't answer back".[8]
Once the lineup was decided, the members flew to Italy to become acquainted with each other.[9] Stevens remarked that the group "felt comfortable with each other from the beginning".[9] Several members of the group said that the "S" in S Club 7 stands for Simon, after the group's creator, although the official explanation of the meaning has always been ambiguous.[10] At one point they were nearly called "Sugar Club" instead of the name that stuck.[11] Another theory is that the group is so-named because "S" is the first letter of the word "seven". McIntosh, in a December 2012 interview, said a lot of Fuller's success has been based on the number 19 (owning 19 Entertainment); therefore as "S" is the 19th letter of the alphabet, the "S" was put into S Club 7.[12]
1999–2000: Debut and success
S Club 7 rose to fame in a children's television series, Miami 7, first broadcast on CBBC between April and July 1999.[13][14] The members played fictional versions of themselves and went on adventures and performed a song in each episode.[13][14] In the US, the show was renamed S Club 7 in Miami and broadcast on Fox Family.[14] By 2000, it had been sold to 100 countries and viewed by 90 million people.[15] The use of the television show to publicise the group was likened to the Monkees.[16][17][18]S Club 7 released their debut single, "Bring It All Back", on 9 June 1999, which The Guardian likened to the Jackson 5.[14] It reached number one on the UK Singles Chart and was certified platinum.[19][20][21] S Club 7's debut album, S Club, was released in October 1999. It reached number two in the UK Albums Chart[20] and was certified double platinum.[19] In 2001, the group released a fashion line targeted at teenagers.[22] In September 1999, "S Club Party" entered the UK charts at number two and number one in New Zealand.[20][23] The double A-side with the ballad "Two in a Million" and the uptempo "You're My Number One", reached number two in the UK charts in December 1999.[20]
In February 2000, S Club 7 won for British Breakthrough Act award at the 2000 BRIT Awards.[24][25] Hasbro announced a licensing deal to create S Club 7 dolls.[26] In April 2000, S Club's second television series, L.A. 7 (renamed S Club 7 in L.A. in the US), began airing.[27][28] Later that year, S Club 7 starred in the television series S Club 7 Go Wild!, in which they travelled the world raising awareness of endangered species.[14][29] "Reach", another retro-styled uptempo track, was released as a single in May 2000 and reached number two on the UK charts.[20]S Club 7's second album, 7, was released on 12 June 2000. It reached number one in the UK charts[15][20] and was certified triple platinum in the UK[19][30] and gold in the US.[31] The second single from the album, "Natural", reached number three in September 2000.[20] That October, S Club 7 launched the annual Poppy Appeal campaign with Dame Thora Hird.[32][33] Alongside numerous other artists, they contributed vocals to a cover of the Rolling Stones song "It's Only Rock 'N' Roll" for Children's Promise, an alliance of seven children's charities.[citation needed] It entered the UK charts at number nineteen.[34] In November 2000, S Club 7 recorded the ballad "Never Had a Dream Come True", the year's official song for the Children in Need charity campaign, which became a number one in the UK and a top-ten hit in the US.S Club 7 released their third studio album, Sunshine, on 26 November 2001. The lead single, "Don't Stop Movin'", released in April 2001, marked a more sophisticated sound, likened to the 1983 Michael Jackson song "Billie Jean".[35] It reached number one,[20] went platinum[19] and became the seventh-best-selling single of 2001.[36] It won The Record of the Year[37] and earned S Club 7 their second BRIT Award, this time for best British single.[38] S Club 7 donated more than £200,000 to Children in Need after it reached number one.[39]On 20 March 2001, Cattermole, Lee and McIntosh were caught with cannabis in Covent Garden, London. They were cautioned by police at Charing Cross Police Station and released without charge.[41] They released an apology through their publicity firm.[42][43] BT and Cadbury, who had sponsorship deals with S Club 7, said they were "disappointed" but kept their contracts with the band,[42] while the Quaker Oats Company ended talks with 19 Entertainment.[44] Pepsi signed a sponsorship deal with S Club 7 within a month of the caution.[45][46] PMS, a merchandising company, sued the group for £800,000, claiming that the drugs scandal ruined its sales.[47]
After spending most of early 2001 rehearsing, the S Club Party 2001 tour began on 19 May 2001.[7] Once the tour was over, the group flew to the United States to film the third series of their television show Hollywood 7.[7] The group had to continuously cope with intense schedules and early starts whilst recording for the programme.[7]
Cattermole and Spearritt began dating in 2001 and were in a relationship until 2008.[48] The relationship was kept secret for the first six months. Cattermole later said it had been "forced" by management and did not happen "organically", as their romance had been written into Hollywood 7.[49] Hollywood 7 aired alongside a new CBBC reality show, S Club Search, which invited children to extend the S Club brand and audition to form a younger version of the band. The new group were to be chosen to sing with S Club 7 on Children in Need 2001 and to tour with them on their future S Club 7 Carnival 2002 tour.[50] The winning children formed a new group, S Club Juniors, and had six top-ten UK hits.[51][52][53]
In November, they recorded a second Children in Need single, "Have You Ever", co-written by Chris Braide and Cathy Dennis. The performance on the night featured many primary school children who had pre-recorded their own versions of the chorus, including the first television appearance by S Club Juniors.[54] The single became the S Club 7's fourth number one[20] and the 21st-biggest-selling single of 2001.[36]
2002: Cattermole's departure
In January 2002, S Club 7 embarked upon their second arena tour.[55]
The third single from Sunshine, "You", reached number two in the UK.[20] The group was disappointed by the choice of single, as they had hoped to continue the more contemporary sound established by "Don't Stop Movin'". Cattermole said later that they had "wanted to be perceived as cool since the very beginning" but that their management had "taken it away".[35]
In 2002, Cattermole quit S Club 7 to rejoin his school metal band, Skua.[48] He later described frustration with the S Club 7 management, and said that "things were being handled so badly, I had to go".[35] Cattermole stayed with S Club 7 until June 2002, featuring in four episodes of their final television series, Viva S Club. His final concert before his departure was Party at the Palace, part of Queen Elizabeth II's Golden Jubilee celebrations.[56] Skua failed to secure a record deal.[35]
After Cattermole's departure, the group was renamed S Club. The remaining members re-signed their deal with Fuller and their record company.[57] "Alive", S Club's first single without Cattermole, reached number five on the UK Singles Chart,[20] and their fourth album, Seeing Double, reached number 17.[20] O'Meara announced that she would step back from performing for the remainder of the year due to a back condition.[58][59]
In April 2003, S Club released their first feature film, Seeing Double, a children's fantasy, in which the group fought the evil scientist Victor Gaghan in his quest to clone the world's pop stars. The film's release was marked by rumours that the group were about to split, which they denied.[60][61]
According to Spearritt, of the €75 million the band had made for Fuller, the band members had received €150,000 each a year. The group members travelled economy on flights and prepared their own food and laundry when working abroad. Spearritt's parents reportedly hired lawyers to collect payments owed to them by Fuller and his management companyTen days after the release of Seeing Double, during their S Club United tour on 21 April 2003, S Club announced on stage that they were breaking up.[58] They cited a mutual split, saying it was time "to move on and to face new challenges".[58] Spearritt said that "she and her fellow bandmates wanted to do their own thing".[64] Fans expressed anger after S Club had denied rumours of a split only weeks prior.[65] Their final single was a double A-side, coupling "Love Ain't Gonna Wait for You", from their fourth album Seeing Double, with a new ballad, "Say Goodbye", released on 26 May, which reached number two in the UK.[20] On 2 June, a greatest hits album, Best: The Greatest Hits of S Club 7, was released, containing the previously unreleased track, "Everybody Get Pumped". It reached number two in the United Kingdom.[20]2003–2008: Solo projects
In 2003, Stevens signed a £1.5 million four-album deal with Polydor and Fuller.[66][67] Her debut solo album, Funky Dory, reached number nine on the UK Albums Chart and was certified gold.[68]
In 2004, Spearritt acted in the films Agent Cody Banks 2: Destination London and Seed of Chucky, and played the lead role of Abby Maitland for five series of the ITV science fiction series Primeval.[69]
In January 2007, O'Meara joined the fifth series of the British reality television show Celebrity Big Brother, broadcast on Channel 4. She and other contestants Danielle Lloyd and Jade Goody were accused of making racist and bullying comments about the Indian contestant Shilpa Shetty, including saying that Indians were thin because they were "sick all the time" as a result of undercooking their food. The incident drew record numbers of viewer complaints and international media coverage.[70][71] After leaving the show, O'Meara denied that she was racist and said that the show's editing had misconstrued her behaviour.[72]
In 2008, O'Meara, Cattermole and McIntosh began performing at British nightclubs, universities and holiday camps as S Club 3, joined occasionally by other band members.[69][35] Cattermole later declared bankruptcy and sold his 2000 BRIT Award at auction.[48] He worked various other jobs, such as a community radio manager and as a psychic.[48]
2014–2015: Reunion
On 14 November 2014, S Club 7 reunited for a BBC Children in Need telethon, performing a medley of "S Club Party", "Reach", "Bring It All Back" and "Don't Stop Movin'".[73][74][75][76]
On 28 April 2015, S Club 7 reissued Best: The Greatest Hits of S Club 7, with a previously unreleased song, "Rain", as well as "Friday Night" from the S Club album.[77][78] In May, S Club 7 performed in UK arenas on the Bring It All Back 2015 tour.[79][80] By December 2017, the group had become a trio again and released a single, "Family".[81]
In 2019, Cattermole said that Fuller, not the group members, had been signed to Polydor as S Club 7. The members were merely affiliates and received "pittance" from merchandise.[35][82] In January 2023, Spearritt said she had been forced into homelessness, living in four temporary homes over a six-month period.[83][62] She said that people wrongly assumed the members of S Club 7 were millionaires, and that they had not been on a good wage compared to the modern music industry.[84]
2023–present: Second reunion and death of Cattermole
On 13 February 2023, S Club 7 announced on The One Show that they would reunite for a two-week 25th-anniversary tour that October.[85][86] On 6 April, at age 46, Cattermole died of several heart conditions: cardiac arrhythmia, acute myocardial ischemia and severe coronary artery atheroma and intraplaque haemorrhage.[87][88] Following his death, Best: The Greatest Hits of S Club 7 reached number seven on the Official Album Downloads Chart Top 100.[89] On 14 May, the group announced they had (for a second time) changed their name to S Club and that Spearritt would not join them on their coming tour, which they would honour as the Good Times Tour as a tribute to Cattermole.[90][91] Though Lee said Spearritt remained a member of the group, other sources reported that she had left.[92] On 30 June "Don't Stop Movin'" entered the vinyl charts at number one.[93]
On 26 July, S Club released their first new single in more than 20 years, "These Are the Days", in memory of Cattermole. It was written and produced by S Club's original songwriting team; Barrett described it as "a nod to our old sound". Spearritt did not take part, saying she was still too upset by Cattermole's death.[94][95][96][97][98]
The band partnered with the British Heart Foundation to raise awareness of heart conditions and raise funds for research.[99][100] The Good Times Tour began on 12 October at the Manchester Arena, with the group paying tribute to Cattermole. A review noted that the members may have been lip syncing during some of their UK tour performances.[101] The group also announced their first ever North American tour as a second leg of their Good Times tour,[102][103][104] which was postponed to early 2024.[105][106]
S Club performed at the 2023 Jingle Bell Ball on 10 December.[107] While attending, they announced new music would be a "definite" in 2024 and the hope for a new television series.[108] The following month, they further discussed the potential for new music, with Lee stating: "If they want to hear more music, we'll do more music."[109]
On 7 March 2024, which is Cattermole's birthday, a new version of the song, "Good Times" featuring McIntosh and Lee on lead vocals, was released on digital platforms.[citation needed]
Musical style
The style of music S Club 7 has is usually pop, or more specifically bubblegum pop. Their first two singles had vocals shared equally amongst the seven members of the group, and it was not until their third single, "Two in a Million", that O'Meara became known as their lead vocalist. Although the band were to progressively change their style over the four years they were together, even their first album had many tracks atypical of the pop genre: "You're My Number One" and "Everybody Wants Ya" were Motown-driven, whereas "Viva La Fiesta" and "It's a Feel Good Thing" were both bouncy, salsa-driven Latino songs. Over the years their style and direction changed progressively with each new album. Their second album 7 had songs with styles that somewhat opposed the traditional pop songs that rival pop bands of the nineties were releasing. With the release of "Natural" in 2000, S Club 7 showcased a new R&B-lite sound. The release of their third album, Sunshine, gave audiences their biggest change: the album contained tracks such as the disco-influenced "Don't Stop Movin'" and the R&B ballad "Show Me Your Colours". The album marked a more mature approach for the band. After the departure of Cattermole, S Club released their fourth and final album, Seeing Double, including the single "Alive", which was called a power-packed dance floor filler. The single has a style similar to that of their final single, "Love Ain't Gonna Wait for You". The album contained dance tracks that varied from their original bubblegum pop stylings, such as the sex for the CBBC generation on "Hey Kitty Kitty".
A reviewer for The Guardian, referring to "Gangsta Love", said "S Club's spiritual home is the suburban disco, not urban underground clubs, and their attempt to go garage on "Gangsta Love" ends up amusing rather than authentic".[110]
'''
wiki.iloc[692, 2] = text

In [7]:
text = '''Yasiin Bey (/jæˈsiːn ˈbeɪ/; born Dante Terrell Smith, December 11, 1973), formerly known as Mos Def (/ˌmoʊs ˈdɛf/), is an American rapper. A prominent figure in conscious hip hop, he is recognized for his use of wordplay and commentary on social and political issues, such as police brutality, American exceptionalism, and the social status of African Americans.
He launched his hip hop recording career in 1994 and joined his siblings in the short-lived rap group Urban Thermo Dynamics (UTD), and guest appeared on albums by Da Bush Babees and De La Soul. In 1996, he and fellow Brooklyn-based rapper Talib Kweli formed the duo Black Star, whose debut album Mos Def & Talib Kweli Are Black Star (1998) contained the singles "Definition" and "Respiration" (featuring Common). He released his solo debut, Black on Both Sides in 1999, which was followed by The New Danger (2004), True Magic (2006), and The Ecstatic (2009).[4] His 2000 single, "Oh No" (with Pharoahe Monch featuring Nate Dogg) remains his sole entry on the Billboard Hot 100 as a solo act.[5] In 2014, About.com listed him 14th on its "50 Greatest Rappers of All Time".[6]
A former child actor in television films, sitcoms, and theater, Bey has appeared in the films Something the Lord Made, Next Day Air, The Hitchhiker's Guide to the Galaxy, 16 Blocks, Be Kind Rewind, The Italian Job, The Woodsman, Bamboozled, and Brown Sugar and in television series such as Dexter and House.[7] He hosted Def Poetry Jam from 2002 to 2007.Yasiin Bey was born Dante Terrell Smith on December 11, 1973, in Brooklyn, New York City,[8] the son of Sheron Smith and Abdulrahman Smith.[9] The eldest of 12 children and step-children, he was raised by his mother in Brooklyn, while his father lived in New Jersey.[10]
His father was initially a member of the Nation of Islam and later followed Imam Warith Deen Mohammed, who merged into the mainstream Sunni Islam from the Nation of Islam. Bey was not exposed to Islam until the age of 13. He is close friends with fellow Muslim hip-hop artists Ali Shaheed Muhammad and Q-Tip.[9]
Bey attended middle school at Philippa Schuyler Middle School in Bushwick, Brooklyn, where he developed his love for acting.[10][11] After returning from filming You Take the Kids in Los Angeles, and getting into a relationship with an older girl, Bey dropped out of high school during sophomore year.[10] Growing up in New York City, during the crack epidemic of the 1980s and early 1990s, he has spoken about witnessing widespread instances of gang violence, theft, and poverty in society, which he largely avoided by working on plays, Off-Off-Broadway and arts programs.[11] In a particularly traumatic childhood experience, Bey witnessed his then five-year-old younger brother, Ilias Bey (born Denard Smith), get hit by a car. Ilias later adopted the alias DCQ and was described by Bey as "my first partner in hip hop".[11]Bey began his music career in 1994, forming the group UTD (or Urban Thermo Dynamics) along with younger brother DCQ and high-school friend Ces.[12] In 2004, they released the album Manifest Destiny, their first and only release to date. The album features a compilation of previously unreleased and re-released tracks recorded during the original UTD run.[13]
In 1996, Bey emerged as a solo artist and worked with De La Soul and Da Bush Babees, before he released his own first single, "Universal Magnetic" in 1997.[14][15]Bey signed with Rawkus Records and formed the rap group Black Star with Talib Kweli.[16] The duo released an album, Mos Def & Talib Kweli Are Black Star, in 1998.[17] Mostly produced by Hi-Tek, the album featured the singles "Respiration" and "Definition", which both reached in the Billboard Hot R&B/Hip-Hop Songs chart.[18]1999–2006: Solo career and various projects
Bey released his solo debut album Black on Both Sides in October 1999, also through Rawkus.[19][20] The single "Ms. Fat Booty" charted,[18] while the album reached #25 on the Billboard 200.[21] Around this time, he also contributed to the Scritti Politti album Anomie & Bonhomie.[22][23]

Yasiin Bey performing at Rock the Bells in New York
In January 2002, Rawkus Records was taken over by Geffen Records,[24] which released his second solo album The New Danger in October 2004.[25] It included contributions by Shuggie Otis and Bernie Worrell, Doug Wimbish, and Will Calhoun as the Black Jack Johnson Band.[26] The album reached #5 on the Billboard 200, making it the most successful for the artist to date.[21] The single "Sex, Love & Money" charted,[18] and was nominated for a Grammy Award.[27] Bey's final solo album for Geffen Records, titled True Magic, was released in 2006.[28]2007–2011: Good Music era and name change
On November 7, 2007, Bey performed live in San Francisco at The Mezzanine venue. The performance was recorded for an upcoming "Live in Concert" DVD. During the event, he announced that he would be releasing a new album to be called The Ecstatic.[29] He performed a number of new tracks; in later shows, he previewed tracks produced by Madlib and was rumored to be going to Kanye West for new material. Producer and fellow Def Poet Al Be Back revealed he would be producing on the album as well.[30] The album was released on June 9, 2009; but only Madlib's production had made the cut, along with tracks by Preservation, The Neptunes, Mr. Flash, Madlib's brother Oh No, J. Dilla, and Georgia Anne Muldrow.
Bey appears alongside Kanye West on the track "Two Words" from The College Dropout album, the track "Drunk and Hot Girls" and the bonus track "Good Night" off West's third major album, Graduation. In 2002, he released the 12" single Fine, which was produced by West and featured on the Brown Sugar soundtrack.In 2007, Bey appeared on Stephen Marley's debut album Mind Control on the song "Hey Baby". He also appeared on the debut album from Apollo Heights on a track titled "Concern". That October, he signed a deal with Downtown Records and appeared on a remix to the song "D.A.N.C.E." by Justice.[32]
In April 2008, he appeared on the title track for a new album by The Roots titled Rising Down. The single, "Life In Marvelous Times", was made officially available through iTunes on November 4, 2008, and was also available for stream on the Roots' website Okayplayer.
In 2009, he worked with Somali-Canadian rapper K'naan to produce the track "America" for K'naan's album Troubadour.[33]
In April 2009, Bey traveled to South Africa for the first time, and performed with The Robert Glasper Experiment at the renowned Cape Town International Jazz Festival. He treated the South African audience with an encore, introduced by his own rendition of John Coltrane's A Love Supreme, followed by a sneak preview of the track "M.D. (Doctor)".[34]
Bey has designed two pairs of limited edition Converse shoes. The shoes were released through Foot Locker stores on August 1, 2009, in limited amounts.[35]
In late 2009, he created his own clothing line with the "UNDRCRWN" brand called the "Mos Def Cut & Sew Collection". The items were released in select U.S. stores and almost exclusively on the UNDRCRWN website.[36] That year, he was among the MCs aligned with American entrepreneur Damon Dash's DD172 and collaborating with American blues rock band the Black Keys on the Blakroc album, a project headed by the Black Keys and Damon Dash.[37] Bey appeared with Harlem-bred rapper Jim Jones and the Black Keys on the Late Show with David Letterman to perform the Blakroc track "Ain't Nothing Like You (Hoochie Coo)".
In 2010, Bey featured on the first single, "Stylo", from the third Gorillaz album, Plastic Beach, alongside soul legend Bobby Womack. He also appeared on the track titled "Sweepstakes". In March 2010, Bey's song "Quiet Dog Bite Hard" was featured in Palm's "Life moves fast. Don't miss a thing." campaign.[38]
In September 2010, after appearing on Kanye West's G.O.O.D. Friday track "Lord Lord Lord", Bey confirmed he had signed with GOOD Music.[39] Bey has been an active contributor to the recovery of the oil spill in the Gulf, performing concerts and raising money towards repairing its damages. In June 2010, he recorded a cover of the classic New Orleans song originally by Smokey Johnson, "It Ain't My Fault" with the Preservation Hall Jazz Band, Lenny Kravitz and Trombone Shorty.In September 2011, the rapper announced that he legally changed his name to Yasiin Bey and would go by that name, retiring his Mos Def moniker.[40][41] Bey told a reporter "I began to fear that Mos Def was being treated as a product, not a person, so I’ve been going by Yasiin since ’99. At first it was just for friends and family, but now I'm declaring it openly."[42] Bey also stated "Mos Def is a name that I built and cultivated over the years, it's a name that the streets taught me, a figure of speech that was given to me by the culture and by my environment, and I feel I've done quite a bit with that name. ...it's time to expand and move on."[43] Yasiin is a name in the Qur'an's 36th surah and was important to the rapper who converted to Islam in his teens. Bey said the name change would eliminate "having any moniker or separation between the self that I see and know myself as."[43]
Shortly after the name change announcement, Bey recorded as the narrator of the children's hip hop musical, Pacha's Pajamas: A Story Written By Nature.
2012–present: Later career and retirement
In January 2012, it was reported that Bey and Talib Kweli had begun "to resurrect" Black Star.[44] In 2015, Bey was featured on A$AP Rocky's second studio album At. Long. Last. ASAP, on the track "Back Home", alongside Acyde and the deceased A$AP Yams.[45] Bey again revived his Mos Def moniker for two new songs in August 2015, titled "Basquiat Ghostwriter" and "Sensei on the Block".[46][47]
On January 19, 2016, Bey announced his retirement from both the music and film industries on Kanye West's website: "I'm retiring from the music recording industry as it is currently assembled today, and also Hollywood, effective immediately. I'm releasing my final album this year, and that's that."[48] After announcing his retirement, he expressed gratitude to everyone who has supported him over the years and revealed his intention to enter the fashion industry and complete a handful of films.[49] Bey also confirmed he still planned to release a collaborative project with Ferrari Sheppard called Dec 99th.[49]
In October 2016, a planned concert in London was cancelled due to travel restrictions imposed on the artist;[50] other European dates suffered the same fate.[51] On October 14, Bey posted a video to Facebook where he announced that he was still planning to retire: "I'm retiring for real this year, this week. With the 17th anniversary of Black on Both Sides being released, I am grateful to have had the career that I have been able to enjoy."[52][53] He also announced one last concert which he said he would be live streaming from Cape Town, South Africa.[54] In November 2016, he was granted the ability to leave but not reenter South Africa and was put on "South Africa's 'undesirable persons' list". He said he would perform one show in Harlem and three in Washington D.C. after leaving South Africa. He announced that Dec 99th, his final album, would be released on December 9. He released three singles from it, "Local Time", "N.A.W." and "Seaside Panic Room".[55] On December 5, 2016, he announced the title of two albums, Negus in Natural Person and As Promised, the latter of which is a collaboration with Southern hip hop producer Mannie Fresh, initially titled OMFGOD.[56] In 2017, his claims of retirement were seen as questionable as he played shows on September 13, at the Fox Theater in Oakland as part of Black Star and performed on Gorillaz' Humanz World Tour, performing "Stylo" with Peven Everett, who filled in for Bobby Womack, who died in 2014. He also has yet to release or announce a release date for the last two albums of his career.[57][58]
In February 2018, Bey announced, on stage, a new Black Star studio album, produced by Madlib, would be released in 2018. However, no specific release date was made available. Later on that year, Bey appeared on the titular track of Kids See Ghosts, the collaborative effort of Kanye West and Kid Cudi. In March 2019, he debuted his album, ንጉሥ (pronounced Negus) at a listening session as part of Art Basel Hong Kong. In a press release, he said that the album "will continue to unfold as a series of varied installations around the world". He has no plans of releasing the album digitally or physically.[59]
On June 28, 2019, Bey appeared on Bandana, an album by Freddie Gibbs and Madlib, alongside Black Thought, on a track named "Education". In November 2019, he stated that unreleased music, such as his collaborative album with Mannie Fresh, was still "on deck" for release.[60] In 2021, he launched "The Midnight Miracle" podcast on Luminary, along with Dave Chappelle and Talib Kweli.[61] In April 2022, a release date for the long-awaited Black Star project was officially announced. The album titled No Fear of Time was released via Luminary on May 3, 2022.[62]
Acting career
Beginnings as child actor
Prior to his career in music, Bey entered public life as a child actor, having played roles in television movies, sitcoms and theater, some of which were under the name Dante Beze.[63][64] At the age of 14, he appeared in the TV movie God Bless the Child, starring Mare Winningham, which aired on ABC in 1988.[65] He played the oldest child in the 1990 family sitcom You Take the Kids, shortly before it was cancelled. In 1995, he played the character Dante, Bill Cosby's sidekick on the short-lived detective show The Cosby Mysteries. In 1996, he starred in a Visa check card commercial featuring Deion Sanders. He also had a small role alongside Michael Jackson in his short film and music video Ghosts (1996).
Feature films, theater and television
After brief appearances in Bamboozled and Monster's Ball, Bey played a rapper who is reluctant to sign with a major label in Brown Sugar. He was nominated for an Image Award and a Teen Choice Award.[66]
In 2001, he took a supporting role to Beyoncé Knowles and Mehki Phifer in the MTV movie Carmen: A Hip Hopera as Lt. Miller, a crooked cop.
In 2002, he played the role of Booth in Suzan-Lori Parks' Topdog/Underdog, a Tony-nominated and Pulitzer-winning Broadway play. He and co-star Jeffrey Wright won a Special Award from the Outer Critics Circle Award for their joint performance.[67] He played Left Ear in the 2003 film The Italian Job. That same year, he appeared in the Alicia Keys music video You Don't Know My Name.
In television, Bey has appeared on NYPD Blue,[68] Comedy Central's Chappelle's Show, and hosted the award-winning HBO spoken word show Def Poetry.[69] He also appeared on the sitcom My Wife and Kids as the disabled friend of Michael Kyle (Damon Wayans).
Bey won "Best Actor, Independent Movie" at the 2005 Black Reel Awards for his portrayal of Detective Sgt. Lucas in The Woodsman. For his portrayal of Vivien Thomas in HBO's film Something the Lord Made, he was nominated for an Emmy[70] and a Golden Globe, and won the Image Award. He also played a bandleader in HBO's Lackawanna Blues. He then landed the role of Ford Prefect in the 2005 movie adaptation of The Hitchhiker's Guide to the Galaxy.

Yasiin Bey and Bruce Willis on the set of 16 Blocks, filmed on location in Chinatown, Manhattan
In 2006, Bey appeared in Dave Chappelle's Block Party alongside Black Star partner Talib Kweli, while also contributing to the film's soundtrack. He was also featured as the banjo player in the Pixie Sketch" from Chappelle's Show: The Lost Episodes, though his appearance was edited out of the DVD. He starred in the action film 16 Blocks alongside Bruce Willis and David Morse. He also landed a recurring guest role on Boondocks, starring as Gangstalicious. He is set to be in Toussaint, a film about Haitian revolutionary Toussaint Louverture, opposite Don Cheadle and Wesley Snipes.[71] He made a cameo appearance as himself in the movie Talladega Nights: The Ballad of Ricky Bobby.
In 2007, Bey narrated the PBS-broadcast documentary Prince Among Slaves.
In 2008, Bey starred in the Michel Gondry movie Be Kind Rewind, playing a video rental store employee whose best friend is played by co-star Jack Black. He also portrayed Chuck Berry in the film Cadillac Records, for which he was nominated for a Black Reel Award and an Image Award.
In 2009, he appeared in the House episode titled "Locked In" as a patient suffering from locked-in syndrome. His performance was well received, with E! saying that Mos Def "delivers an Emmy-worthy performance".[72] He was also in the 2009 film Next Day Air.
In 2010, he appeared on the children's show Yo Gabba Gabba! as Super Mr. Superhero. He also appeared in A Free Man of Color, John Guare's play at the Vivian Beaumont Theatre.[73]
In 2011, he began a multi-episode appearance on the sixth season of Showtime television series Dexter. He played Brother Sam, an ex-convict who has supposedly found religion despite finding himself in violent situations.[74]
In January 2016, Bey announced his retirement from the music and the film industries on Kanye West's website. In March 2016, it was announced that he had been attached to star in "his last live-action film", The Disconnected, a science fiction thriller dealing with policing, identity, and the intersection of technology and humanity.[75]
Social and political views
In 2000, paired with Talib Kweli, Bey organized the Hip Hop for Respect project to speak out against police brutality. It was created in response to the 1999 police shooting of Amadou Diallo, and sought to accumulate 41 artists to the roster, one to match each of the 41 gunshots fired on Diallo.
Bey is well known for his leftist activism.[76] In 2000, he performed a benefit concert for death row inmate Mumia Abu-Jamal.[77] In May 2005, Bey, Talib Kweli, R&B singer Martin Luther and City Councilman Charles Barron approached New York City Hall, demanding the withdrawal of the $1 million bounty for Assata Shakur.[78]
In September 2005, Bey released the single "Katrina Clap", renamed "Dollar Day" for True Magic (utilizing the instrumental for New Orleans rappers UTP's "Nolia Clap"). The song is a criticism of the Bush administration's response to Hurricane Katrina. On the night of the MTV Video Music Awards, Bey pulled up in front of Radio City Music Hall on a flatbed truck and began performing the "Katrina Clap" single in front of a crowd that quickly gathered around him. He was subsequently arrested for not having a public performance permit in his possession.[79]
In October 2006, Bey appeared on 4Real, a documentary television series.[80] Appearing in the episode "City of God", he and the 4Real crew traveled to City of God, a favela in Rio de Janeiro, Brazil, to meet Brazilian MC MV Bill and learn about the crime and social problems of the community.[81]
On September 7, 2007, Bey appeared on Real Time with Bill Maher where he spoke about racism against African Americans, citing the government response to Hurricane Katrina, the Jena Six case, and the murder conviction of Mumia Abu-Jamal. He appeared on Real Time again on March 27, 2009, and spoke about the risk of nuclear weapons.[82]
In September 2011, Bey joined the cast of the environmental children's hip hop musical Pacha's Pajamas: A Story Written By Nature as narrator. In July 2013, he appeared in a short film released by the human rights organization Reprieve, depicting the forced-feeding methods used at the Guantanamo Bay detention camps. This transpired after a document containing the military instructions for the procedure was leaked.[83][84]
In September 2018, Bey and advertising executive Free Richardson opened an art exhibition to the public in an art gallery in the South Bronx, called the Compound, centered around hip-hop and fine art. The goal of this gallery was to help bridge the gap between the two fields, by showcasing artists from marginalized backgrounds who normally would not be represented in art galleries. Art forms with negative connotations, such as graffiti, as presented in a more optimal context. "The purpose of the gallery is to say all art is equal," Mr. Richardson said. "But we are in the borough that created hip-hop, which is the biggest art form in the world, so it's always an extending arm. It's always present."[85]
Legacy
About.com ranked him #14 on its list of the Top 50 MCs of Our Time,[6] and The Source ranked him #23 on their list of the Top 50 Lyricists of All Time.[86] AllMusic called him one of the most promising rappers to emerge in the late 1990s,[87] and one of hip-hop's brightest hopes entering the 21st century.[19] He has influenced numerous hip hop artists throughout his career, including Lupe Fiasco, Jay Electronica, Kid Cudi, and Saigon, and Logic.[88][89][90] Kendrick Lamar once mentioned him as a very early inspiration as a young rapper, though Lamar denied being a part of the conscious rap movement.[91] He has also directly influenced artists outside of hip-hop, including English singer-songwriters Adele and Amy Winehouse.[92][93]
Personal life
Bey married Maria Yepes in 1996. He filed for divorce from Yepes in 2006.[94] She took him to court over failure in child-support obligations, paying $2,000 short of the monthly $10,000 he is ordered to pay for their two daughters.[95][96] He has four other children.[97]
His mother Sheron "Umi" Smith has actively managed portions of her son's career.[98] She is a motivational speaker, and authored the book Shine Your Light: A Life Skills Workbook, where she details her experience as a single mother raising him.[99]
In January 2016, Bey was ordered to leave South Africa and not return for five years, having stayed in the country illegally on an expired tourist visa granted in May 2013.[100] That month, he was charged with using an unrecognized World Passport and having lived illegally in South Africa since 2014.[101][102] He had reportedly recruited Kanye West to help defend him, and posted a message on West's website announcing his retirement from show business.[103][104] There was an ongoing court case in relation to immigration offenses involving the artist and his family.[105][106] He was allowed to leave South Africa on November 22, 2016, but he was barred from coming back.[107]
'''
wiki.iloc[615, 2] = text

In [8]:
nationality = pd.read_csv('/Users/vincenzocamerlengo/Desktop/Data Visualization/Project/Cartella Condivisa/wordlcoud/further_updated_nationalities_artists-2.csv')

In [9]:
nationality = nationality.rename(columns={'name': 'Artist'})

In [10]:
wiki.shape

(977, 3)

# Merging Wikipedia Data with Existing Datasets
The fetched Wikipedia data is merged with our pre-existing datasets. This step is crucial as it combines the newly acquired information with our original data, enhancing the richness of our dataset and ensuring that all related data points are aligned.

In [11]:
wiki = pd.merge(wiki, nationality, on='Artist', how='inner')

In [ ]:
wiki.head()

,Artist,Genre,full_text,2023_visits,nationality
0,Taylor Swift,pop music,"Taylor Alison Swift (born December 13, 1989) i...",22030018,United States of America
1,XXXTentacion,punk rock,"Jahseh Dwayne Ricardo Onfroy (January 23, 1998...",20104479,United States of America
2,Alia Bhatt,Unspecified,Alia Bhatt (; born 15 March 1993) is a British...,12669342,India
3,Tina Turner,rock and roll,Tina Turner (born Anna Mae Bullock; November 2...,11590403,United States of America
4,Keanu Reeves,hip hop music,Keanu Charles Reeves ( kee-AH-noo; born Septem...,9916477,Canada


# Setting Up Country and Continent Classifications
This code section categorizes each artist by country and continent based on predefined lists. This geographical categorization aids in subsequent regional analyses.

In [12]:

wiki['nationality'].replace('Punjab', 'India', inplace=True)
wiki['nationality'].replace('Canadian-American', 'Canada', inplace=True)
wiki['nationality'].replace('Troy', 'Greece', inplace=True)


In [13]:
# Africa
africa = [
    'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cabo Verde', 
    'Cameroon', 'Central African Republic', 'Chad', 'Comoros', 'Democratic Republic of the Congo', 
    'Republic of the Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 
    'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Ivory Coast', 'Kenya', 
    'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 
    'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 
    'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 
    'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe'
]

# Asia
asia = [
    'Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 
    'Cambodia', 'China', 'Cyprus', 'Georgia', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 
    'Japan', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Lebanon', 'Malaysia', 
    'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'North Korea', 'Oman', 'Pakistan', 'Palestine', 
    'Philippines', 'Qatar', 'Saudi Arabia', 'Singapore', 'South Korea', 'Sri Lanka', 'Syria', 
    'Taiwan', 'Tajikistan', 'Thailand', 'Timor-Leste', 'Turkey', 'Turkmenistan', 
    'United Arab Emirates', 'Uzbekistan', 'Vietnam', 'Yemen'
]

# Europe
europe = [
    'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 
    'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Kingdom of Denmark', 'Estonia', 'Finland', 
    'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kosovo', 
    'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 
    'Montenegro', 'Kingdom of the Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 
    'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 
    'Switzerland', 'Ukraine', 'United Kingdom', 'Vatican City', 'Swedish', 'Republic of Ireland',
    'Scotland', 'England', 
]

# North America
north_america = [
    'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada', 'Costa Rica', 
    'Cuba', 'Dominica', 'Dominican Republic', 'El Salvador', 'Grenada', 'Guatemala', 
    'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama', 'Saint Kitts and Nevis', 
    'Saint Lucia', 'Saint Vincent and the Grenadines', 'Trinidad and Tobago', 'United States of America'
]

# South America
south_america = [
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 
    'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

# Australia/Oceania
oceania = [
    'Australia', 'Fiji', 'Kiribati', 'Marshall Islands', 'Micronesia', 'Nauru', 'New Zealand', 
    'Palau', 'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu'
]

In [14]:
import numpy as np
continent = []

for i in range(wiki.shape[0]):
    if wiki['nationality'][i] in africa:
        continent.append('Africa')
    elif wiki['nationality'][i] in asia:
        continent.append('Asia')
    elif wiki['nationality'][i] in europe:
        continent.append('Europe')
    elif wiki['nationality'][i] in north_america:
        continent.append('North America')
    elif wiki['nationality'][i] in south_america:
        continent.append('South America')
    elif wiki['nationality'][i] in oceania:
        continent.append('Oceania')
    else:
        continent.append(np.nan)

wiki['continent'] = continent

In [17]:
coordinates = pd.read_csv('/Users/vincenzocamerlengo/Desktop/Data Visualization/Project/Cartella Condivisa/wordlcoud/artist_birthplaces_with_coordinates.csv')

In [19]:
wiki = pd.merge(wiki, coordinates, on='Artist', how='inner')

In [22]:
#wiki.to_csv('/Users/vincenzocamerlengo/Desktop/Data Visualization/Project/Cartella Condivisa/wordlcoud/FILE_COMPLETO.csv', index=False)